In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../Data/accident_data.csv')

/Users/mattlampl/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (19,20,73,77,78,79,80,81,82,83,84,183,189) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
location_data = data.dropna(subset=['LONGITUDE', 'LATITUDE'])

In [4]:
geometry = geometry = [Point(xy) for xy in zip(location_data['DEC_LONG'], location_data['DEC_LAT'])]
location_gdf = gpd.GeoDataFrame(location_data, geometry=geometry, crs='EPSG:4326')

In [5]:
# Import the roads file
roads = gpd.read_file('alleghenycounty_streetcenterlines202303.zip', crs="EPSG:4326")

In [6]:
acc_road_join = gpd.sjoin_nearest(location_gdf.to_crs(roads.crs), roads, how='left')

In [ ]:
acc_road_join['FATAL']

1         0
2         0
6         0
11        0
14        0
         ..
216619    0
216620    0
216621    0
216622    0
216623    0
Name: FATAL, Length: 206662, dtype: int64

In [ ]:
acc_road_agg = acc_road_join.groupby('FULL_NAME').agg({'CRASH_CRN':'count', 
                                                       'FATAL':'sum', 'INJURY':'sum'
                                                       'FATAL_OR_MAJ_INJ':'sum'})\
    .reset_index()\
    .rename(columns={'FULL_NAME':'STREET', 'CRASH_CRN':'COUNT', 'FATAL':'NUM_FATAL', 'INJURY':'NUM_INJURY'})\
    .sort_values('COUNT', ascending=False)

In [ ]:
acc_road_agg['PCT_FATAL'] = acc_road_agg['NUM_FATAL'] / acc_road_agg['COUNT']
acc_road_agg['PCT_INJURY'] = acc_road_agg['NUM_INJURY'] / acc_road_agg['COUNT']
acc_road_agg.head(20)

,STREET,COUNT,NUM_FATAL,NUM_INJURY,PCT_FATAL,PCT_INJURY
3163,I376 EB,6808,27,2996,0.003966,0.440071
3198,I376 WB,5966,27,2625,0.004526,0.439993
5977,SAW MILL RUN BLVD,3045,15,1344,0.004926,0.441379
4907,OHIO RIVER BLVD,2684,22,1158,0.008197,0.431446
5221,PERRY HWY,2583,15,1145,0.005807,0.443283
73,5TH AVE,2579,11,1223,0.004265,0.474215
6329,STATE ROUTE 28 NB,2259,19,790,0.008411,0.349712
6347,STATE ROUTE 28 SB,2248,22,796,0.009786,0.354093
5195,PENN AVE,2073,10,959,0.004824,0.462615
7226,WILLIAM PENN HWY,2057,9,1074,0.004375,0.522120


In [ ]:
road_acc_counts = roads.merge(acc_road_agg, how='inner', left_on='FULL_NAME', right_on='STREET')

In [ ]:
def to_geojson():
    road_acc_counts.to_file('road_acc_counts.geojson', driver='GeoJSON')